# psiCI: test module

Test the results of configuration-interaction (CI) calculations using the `psiCI` module against known formulas for Hartree-Fock (HF) orbitals.

In [1]:
import psi4
import numpy as np
import psiCI

# Molecular model

Molecuar model we use for checking the `psiCI` module: set a close-shell molecular system and perform a HF ground-state calculation.

In [2]:
# Molecular model
mol = psi4.geometry("""
         0 1
         H   0.865 0 0
         Li -0.865 0 0
         symmetry c1 """)

#mol = psi4.geometry("""
#         0 1
#         O
#         H 1 1.0
#         H 1 1.0 2 104.5
#         symmetry c1 """)

# HF ground state
psi4.set_options({
        'basis':        'cc-pvtz',
        'scf_type':     'pk',
        'reference':    'rhf',
        'mp2_type':     'conv',
        'e_convergence': 1e-9,
        'd_convergence': 1e-9})
#basis = sto-3g

scf_e, wfn = psi4.energy('scf', return_wfn=True)

# CI model

CI model, starting with single excitations (CIS)

In [3]:
# Create the CI-modle object
PCI = psiCI.psiCI(wfn)

# Set the CIS basis (including virtual orbitals 1 and 3)
nMO = int(0.5*PCI.numberElectron)
print("The molecule has " + str(nMO) + " occupied spatial orbitals\n")

MO = np.concatenate((range(1,nMO+2), [nMO+4]))
SO = np.concatenate((-MO,MO))
print("The active spin orbitals are " + str(SO) + "\n\n")

PCI.setConfiguration(mode="CIS",active=SO)

The molecule has 2 occupied spatial orbitals

The active spin orbitals are [-1 -2 -3 -6  1  2  3  6]


################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 4
  * Total spin  = 0.0
  * Reference   = [-1 -2  1  2]
  * Active      = [-1 -2 -3 -6]
                = [1 2 3 6]
  * 9 configurations

################################################################################



We can now proceed with calculating the CI matrix.

In [4]:
CI = PCI.computeCI()

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Electronic structure =======================================================
  * 4 electrons
  * 44 spatial orbitals
  * 9 configurations

=== Configuration-interaction calculation ======================================
  * tolerance: 1e-10
  * 4 active spatial orbitals

=== CI matrix calculation ======================================================
  * Core Hamiltonian                                                        done
  * Two-electron integrals                                                  done
  * CI matrix                                                               done

#####################################################

As a sanity check, we perform a spectrum analysis of the CI matrix.

In [5]:
PCI.analyzeSpectrum(CI,state=np.asarray(range(1,6)))

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== CI matrix spectrum =========================================================
  * Ground state
    Total energy      =     -8.903 a.u. =   -242.255 eV
    > 100.000 % [-1 -2  1  2]

  * Excited state 1
    Total energy      =     -8.777 a.u. =   -238.827 eV
    Excitation energy =      0.126 a.u. =      3.429 eV
    >   7.103 % [-1 -6  1  2]
    >  42.896 % [-1 -3  1  2]
    >  42.896 % [-1 -2  1  3]
    >   7.103 % [-1 -2  1  6]

  * Excited state 2
    Total energy      =     -8.752 a.u. =   -238.143 eV
    Excitation energy =      0.151 a.u. =      4.112 eV
    >   2.193 % [-1 -6  1  2]
    >  47.806 % [-1 -3  1  2]
    >  47.806 % [-1 -2  

# Test the CI results

We now can move to checking the results of our CIS calculation

## Symmetry of the CI matrix

Since we are using real orbitals, we know that the CI matrix must be real symmetric.

In [6]:
print("Symmetry of the CI matrix")
if np.all((CI.T-CI) == 0):
    print("  > Pass")
else:
    print("  > Fail")

Symmetry of the CI matrix
  > Pass


## Reference energy

By construction, we know that the energy of the reference in the CI matrix must match that of the HF calculation

In [7]:
# Recover energies
EHF = scf_e - mol.nuclear_repulsion_energy()
ECI = CI[0,0]

# Display results
print("Check reference energy (a.u.)")
print("  * HF energy = " + str(EHF))
print("  * CI energy = " + str(ECI))
if np.abs(EHF-ECI) < 1e-8:
    print("  > Pass")
else:
    print("  > Fail")

Check reference energy (a.u.)
  * HF energy = -8.90271801769431
  * CI energy = -8.90271801769433
  > Pass


## Brillouin's theorem

For HF orbitals, Brillouin's theorem states that there are no mixing between the HF ground-state and single-excitation wave functions.

In [8]:
print("Brillouin's theorem")
if np.all(np.abs(CI[0,1:]) < 1e-8):
    print("  > Pass")
else:
    print("  > Fail" + str(CI[0,1:]))

Brillouin's theorem
  > Pass


## Single-excitation elements

Single-excitation elements correspond to the coupling between two single excitation configuration states and are given by (assuming real-valued spin orbitals)

$$\langle \psi_a^r | \hat{\mathcal{H}} | \psi_b^s \rangle = E_0 \delta_{ab}\delta_{rs} + F_{rs}\delta_{ab} - F_{ab}\delta_{rs} + \langle rb | as \rangle - \langle rb | sa \rangle$$

where $E_0$ is the HF ground-state energy and with

$$F_{ab} = \langle a | \hat{h} | b \rangle + \sum_{k\in\psi} \langle ak | bk\rangle - \langle ak | kb\rangle$$

First we look at the diagonal elements, where $a=b$ and $r=s$

In [9]:
# Initialization
print("Diagonal single-excitation elements")

mints = psi4.core.MintsHelper(wfn.basisset())
ref   = PCI.configuration[0,:]

C = np.asarray(wfn.Ca())
H = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())
H = C.T @ H @ C

# Check elements
for n in range(1,CI.shape[0]):
    # Initialization
    exc = PCI.configuration[n,:]
    
    # Identify the excitation
    ic = np.intersect1d(ref,exc,assume_unique=True)
    a  = np.setdiff1d(ref,ic,assume_unique=True)
    r  = np.setdiff1d(exc,ic,assume_unique=True)

    aa = np.abs(a)-1
    ar = np.abs(r)-1

    # Calculate the CI matrix element
    Ca = psi4.core.Matrix.from_array(C[:,aa].reshape(-1, 1))
    Cr = psi4.core.Matrix.from_array(C[:,ar].reshape(-1, 1))
    
    E  = CI[0,0] + H[ar,ar] - H[aa,aa]
    E += np.asarray(mints.mo_eri(Cr,Ca,Ca,Cr))[0,0,0,0]                  #<ra|ar> = [ra|ar] (Note: a and r have the same spin)
    E -= np.asarray(mints.mo_eri(Cr,Cr,Ca,Ca))[0,0,0,0]                  #<ra|ra> = [rr|aa]

    for k in ref:
        Ck = psi4.core.Matrix.from_array(C[:,np.abs(k)-1].reshape(-1, 1))
        E += np.asarray(mints.mo_eri(Cr,Cr,Ck,Ck))[0,0,0,0]              #<rk|rk> = [rr|kk]
        E -= np.asarray(mints.mo_eri(Cr,Ck,Ck,Cr))[0,0,0,0] * (r*k > 0)  #<rk|kr> = [rk|kr]
        E -= np.asarray(mints.mo_eri(Ca,Ca,Ck,Ck))[0,0,0,0]              #<ak|ak> = [aa|kk]
        E += np.asarray(mints.mo_eri(Ca,Ck,Ck,Ca))[0,0,0,0] * (a*k > 0)  #<ak|ka> = [ak|ka]

    # Display result
    if np.abs(E - CI[n,n]) < 1e-8:
        print("  > Pass excited configuration " + str(n))
    else:
        print("  > Fail excited configuration " + str(n) + " | error = " + str(E - CI[n,n]))

Diagonal single-excitation elements
  > Pass excited configuration 1
  > Pass excited configuration 2
  > Pass excited configuration 3
  > Pass excited configuration 4
  > Pass excited configuration 5
  > Pass excited configuration 6
  > Pass excited configuration 7
  > Pass excited configuration 8


We now look at the off-diagonal elements, where $a\neq b$ or $r\neq s$

In [10]:
# Check elements
print("Off-diagonal single-excitation elements")
for n in range(1,CI.shape[0]):
    for m in range(n+1,CI.shape[0]):
        # Identify the excitations
        exc = PCI.configuration[n,:]
        ic  = np.intersect1d(ref,exc,assume_unique=True)
        a   = np.setdiff1d(ref,ic,assume_unique=True)
        r   = np.setdiff1d(exc,ic,assume_unique=True)
        
        exc = PCI.configuration[m,:]
        ic  = np.intersect1d(ref,exc,assume_unique=True)
        b   = np.setdiff1d(ref,ic,assume_unique=True)
        t   = np.setdiff1d(exc,ic,assume_unique=True)

        aa = np.abs(a)-1
        ab = np.abs(b)-1
        ar = np.abs(r)-1
        at = np.abs(t)-1
        
        # Calculate the CI matrix element
        Ca = psi4.core.Matrix.from_array(C[:,aa].reshape(-1, 1))
        Cb = psi4.core.Matrix.from_array(C[:,ab].reshape(-1, 1))
        Cr = psi4.core.Matrix.from_array(C[:,ar].reshape(-1, 1))
        Ct = psi4.core.Matrix.from_array(C[:,at].reshape(-1, 1))

        E  = np.asarray(mints.mo_eri(Cr,Ca,Cb,Ct))[0,0,0,0]              #<rb|at> = [ra|bt] (note r and a, and t and b have the same spin)
        E -= np.asarray(mints.mo_eri(Cr,Ct,Cb,Ca))[0,0,0,0] * (a*b > 0)  #<rb|ta> = [rt|ba]

        if (a == b) and (r*t > 0):
            E += H[ar,at]
            
            for k in ref:
                Ck = psi4.core.Matrix.from_array(C[:,np.abs(k)-1].reshape(-1, 1))
                E += np.asarray(mints.mo_eri(Cr,Ct,Ck,Ck))[0,0,0,0]              #<rk|tk> = [rt|kk]
                E -= np.asarray(mints.mo_eri(Cr,Ck,Ck,Ct))[0,0,0,0] * (r*k > 0)  #<rk|kt> = [rk|kt]

        if (r == t) and (a*b > 0):
            E -= H[aa,ab]
            
            for k in ref:
                Ck = psi4.core.Matrix.from_array(C[:,np.abs(k)-1].reshape(-1, 1))
                E -= np.asarray(mints.mo_eri(Ca,Cb,Ck,Ck))[0,0,0,0]              #<ak|bk> = [ab|kk]
                E += np.asarray(mints.mo_eri(Ca,Ck,Ck,Cb))[0,0,0,0] * (a*k > 0)  #<ak|kb> = [ak|kb]
            
        # Display result
        if np.abs(E - CI[n,m]) < 1e-8:
            print("  > Pass excited configuration " + str(n) + ", " + str(m))
        else:
            print("  > Fail excited configuration " + str(n) + ", " + str(m) + " | error = " + str(E - CI[n,m]))

Off-diagonal single-excitation elements
  > Pass excited configuration 1, 2
  > Pass excited configuration 1, 3
  > Pass excited configuration 1, 4
  > Pass excited configuration 1, 5
  > Pass excited configuration 1, 6
  > Pass excited configuration 1, 7
  > Pass excited configuration 1, 8
  > Pass excited configuration 2, 3
  > Pass excited configuration 2, 4
  > Pass excited configuration 2, 5
  > Pass excited configuration 2, 6
  > Pass excited configuration 2, 7
  > Pass excited configuration 2, 8
  > Pass excited configuration 3, 4
  > Pass excited configuration 3, 5
  > Pass excited configuration 3, 6
  > Pass excited configuration 3, 7
  > Pass excited configuration 3, 8
  > Pass excited configuration 4, 5
  > Pass excited configuration 4, 6
  > Pass excited configuration 4, 7
  > Pass excited configuration 4, 8
  > Pass excited configuration 5, 6
  > Pass excited configuration 5, 7
  > Pass excited configuration 5, 8
  > Pass excited configuration 6, 7
  > Pass excited configu

# Test the configuration-state builder

In this section, we test the automated routines for building the configuration-state basis.

## CIS

First we check the configuration state basis for single-reference CIS with user-defined `reference` and `active` spaces

In [11]:
# Configuration state basis
PCI.setConfiguration(mode="CIS",reference=np.array([-1,-2,-3,1,2]),active=np.array([-3,-5,-6,1,3,4,5]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,-3,1,2],
                                     [-5,-2,-3,1,2],[-6,-2,-3,1,2],
                                     [-1,-5,-3,1,2],[-1,-6,-3,1,2],
                                     [-1,-2,-5,1,2],[-1,-2,-6,1,2],
                                     [-1,-2,-3,3,2],[-1,-2,-3,4,2],[-1,-2,-3,5,2],
                                     [-1,-2,-3,1,3],[-1,-2,-3,1,4],[-1,-2,-3,1,5]]),axis=1),axis=0)

# Check results
print("Single-reference CIS")

if PCI.numberElectron == 5:
    print("  > Pass number of electrons")
else:
    print("  > Fail number of electrons")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 5
  * Total spin  = -0.5
  * Reference   = [-1 -2 -3  1  2]
  * Active      = [-3 -5 -6]
                = [1 3 4 5]
  * 13 configurations

################################################################################

Single-reference CIS
  > Pass number of electrons
  > Pass configuration state basis


Now we add constraints on frozen orbitals

In [12]:
# Configuration state basis
PCI.setConfiguration(mode="CIS",reference=np.array([-1,-2,-3,1,2]),active=np.array([-3,-5,-6,1,3,4,5]),frozen=np.array([-2,1]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,-3,1,2],
                                     [-5,-2,-3,1,2],[-6,-2,-3,1,2],
                                     [-1,-2,-5,1,2],[-1,-2,-6,1,2],
                                     [-1,-2,-3,1,3],[-1,-2,-3,1,4],[-1,-2,-3,1,5]]),axis=1),axis=0)

# Check results
print("Single-reference CIS with frozen orbitals")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 5
  * Total spin  = -0.5
  * Reference   = [-1 -2 -3  1  2]
  * Active      = [-3 -5 -6]
                = [1 3 4 5]
  * Frozen      = [-2  1]
  * 8 configurations

################################################################################

Single-reference CIS with frozen orbitals
  > Pass configuration state basis


Add constraints on double ionizations

In [13]:
# Configuration state basis
PCI.setConfiguration(mode="CIS",reference=np.array([-1,-2,-3,1,2]),active=np.array([-3,-5,-6,1,3,4,5]),noDouble=np.array([2,3]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,-3,1,2],
                                     [-1,-5,-3,1,2],[-1,-6,-3,1,2],
                                     [-1,-2,-3,1,4],[-1,-2,-3,1,5]]),axis=1),axis=0)

# Check results
print("Single-reference CIS with constraints on double excitations")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 5
  * Total spin  = -0.5
  * Reference   = [-1 -2 -3  1  2]
  * Active      = [-3 -5 -6]
                = [1 3 4 5]
  * No double   = [2 3]
  * 5 configurations

################################################################################

Single-reference CIS with constraints on double excitations
  > Pass configuration state basis


Add constraints on empty orbitals

In [14]:
# Configuration state basis
PCI.setConfiguration(mode="CIS",reference=np.array([-1,-2,-3,1,2]),active=np.array([-3,-5,-6,1,3,4,5]),noDouble=np.array([2,3]),noEmpty=np.array([3,5]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,-3,1,2],[-1,-5,-3,1,2],[-1,-2,-3,1,5]]),axis=1),axis=0)

# Check results
print("Single-reference CIS with constraints on empty orbitals")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 5
  * Total spin  = -0.5
  * Reference   = [-1 -2 -3  1  2]
  * Active      = [-3 -5 -6]
                = [1 3 4 5]
  * No double   = [2 3]
  * Not empty   = [3 5]
  * 3 configurations

################################################################################

Single-reference CIS with constraints on empty orbitals
  > Pass configuration state basis


Finally, we check the configuration state basis for multi-reference CIS

In [15]:
# Configuration state basis
PCI.setConfiguration(mode="CIS",reference=np.array([[-1,-2,-3,1,2],[-1,-2,-4,1,2]]),active=np.array([-3,-5,-6,1,3,4,5]),frozen=np.array([-1,-2,1]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,-3,1,2],[-1,-2,-4,1,2],
                                     [-1,-2,-5,1,2],[-1,-2,-6,1,2],
                                     [-1,-2,-3,1,3],[-1,-2,-3,1,4],[-1,-2,-3,1,5],
                                     [-1,-2,-4,1,3],[-1,-2,-4,1,4],[-1,-2,-4,1,5]]),axis=1),axis=0)

# Check results
print("Multi-reference CIS")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CIS
  * Nb. elec.   = 5
  * Total spin  = [-0.5 -0.5]
  * References  = [-1 -2 -3  1  2]
                  [-1 -2 -4  1  2]
  * Active      = [-3 -5 -6]
                = [1 3 4 5]
  * Frozen      = [-1 -2  1]
  * 10 configurations

################################################################################

Multi-reference CIS
  > Pass configuration state basis


## CISD

We now perform a similar verification of the configuration state basis building for single-reference CISD model (here testing all the basis options at once)

In [16]:
# Configuration state basis
PCI.setConfiguration(mode="CISD",reference=np.array([-1,-3,1,2,3]),active=np.array([-1,-2,-5,3,5,6]),
                     frozen=np.array([-1,3]),noDouble=np.array([5]),noEmpty=np.array([3,5]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-3,1,2,3],
                                     [-1,-5,1,2,3],[-1,-3,5,2,3],[-1,-3,1,5,3],[-1,-3,5,6,3],
                                     [-1,-2,5,2,3],[-1,-2,1,5,3],
                                     [-1,-5,6,2,3],[-1,-5,1,6,3]]),axis=1),axis=0)

# Check results
print("Single-reference CISD")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Nb. elec.   = 5
  * Total spin  = 0.5
  * Reference   = [-1 -3  1  2  3]
  * Active      = [-1 -2 -5]
                = [3 5 6]
  * Frozen      = [-1  3]
  * No double   = [5]
  * Not empty   = [3 5]
  * 9 configurations

################################################################################

Single-reference CISD
  > Pass configuration state basis


Next, we check multi-reference CISD.

In [17]:
# Configuration state basis
PCI.setConfiguration(mode="CISD",reference=np.array([[-1,-3,1,2,3],[-1,-2,1,2,3]]),active=np.array([-1,-2,-5,3,5,6]),
                     frozen=np.array([-1,1,2]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-3,1,2,3],[-1,-2,1,2,3],
                                     [-1,-5,1,2,3],[-1,-3,1,2,5],[-1,-3,1,2,6],
                                     [-1,-2,1,2,5],[-1,-2,1,2,6],
                                     [-1,-5,1,2,5],[-1,-5,1,2,6]]),axis=1),axis=0)

# Check results
print("Multi-reference CISD")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = CISD
  * Nb. elec.   = 5
  * Total spin  = [0.5 0.5]
  * References  = [-1 -3  1  2  3]
                  [-1 -2  1  2  3]
  * Active      = [-1 -2 -5]
                = [3 5 6]
  * Frozen      = [-1  1  2]
  * 9 configurations

################################################################################

Multi-reference CISD
  > Pass configuration state basis


## RAS

Finally, we check the configuration state basis building for RAS model

In [18]:
# Configuration state basis
PCI.numberElectron = 4
PCI.setConfiguration(mode="RAS",active=np.array([-1,-2,-3,-4,-5,1,2,3,4,5]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,1,2],[-1,-2,1,3],[-1,-2,1,4],[-1,-2,1,5],
                                     [-1,-2,2,3],[-1,-2,2,4],[-1,-2,2,5],
                                     [-1,-2,3,4],[-1,-2,3,5],
                                     [-1,-2,4,5],
                                     [-1,-3,1,2],[-1,-3,1,3],[-1,-3,1,4],[-1,-3,1,5],
                                     [-1,-3,2,3],[-1,-3,2,4],[-1,-3,2,5],
                                     [-1,-3,3,4],[-1,-3,3,5],
                                     [-1,-3,4,5],
                                     [-1,-4,1,2],[-1,-4,1,3],[-1,-4,1,4],[-1,-4,1,5],
                                     [-1,-4,2,3],[-1,-4,2,4],[-1,-4,2,5],
                                     [-1,-4,3,4,],[-1,-4,3,5],
                                     [-1,-4,4,5],
                                     [-1,-5,1,2],[-1,-5,1,3],[-1,-5,1,4],[-1,-5,1,5],
                                     [-1,-5,2,3],[-1,-5,2,4],[-1,-5,2,5],
                                     [-1,-5,3,4],[-1,-5,3,5],
                                     [-1,-5,4,5],
                                     [-2,-3,1,2],[-2,-3,1,3],[-2,-3,1,4],[-2,-3,1,5],
                                     [-2,-3,2,3],[-2,-3,2,4],[-2,-3,2,5],
                                     [-2,-3,3,4],[-2,-3,3,5],
                                     [-2,-3,4,5],
                                     [-2,-4,1,2],[-2,-4,1,3],[-2,-4,1,4],[-2,-4,1,5],
                                     [-2,-4,2,3],[-2,-4,2,4],[-2,-4,2,5],
                                     [-2,-4,3,4],[-2,-4,3,5],
                                     [-2,-4,4,5],
                                     [-2,-5,1,2],[-2,-5,1,3],[-2,-5,1,4],[-2,-5,1,5],
                                     [-2,-5,2,3],[-2,-5,2,4],[-2,-5,2,5],
                                     [-2,-5,3,4],[-2,-5,3,5],
                                     [-2,-5,4,5],
                                     [-3,-4,1,2],[-3,-4,1,3],[-3,-4,1,4],[-3,-4,1,5],
                                     [-3,-4,2,3],[-3,-4,2,4],[-3,-4,2,5],
                                     [-3,-4,3,4],[-3,-4,3,5],
                                     [-3,-4,4,5],
                                     [-3,-5,1,2],[-3,-5,1,3],[-3,-5,1,4],[-3,-5,1,5],
                                     [-3,-5,2,3],[-3,-5,2,4],[-3,-5,2,5],
                                     [-3,-5,3,4],[-3,-5,3,5],
                                     [-3,-5,4,5],
                                     [-4,-5,1,2],[-4,-5,1,3],[-4,-5,1,4],[-4,-5,1,5],
                                     [-4,-5,2,3],[-4,-5,2,4],[-4,-5,2,5],
                                     [-4,-5,3,4],[-4,-5,3,5],
                                     [-4,-5,4,5]]),axis=1),axis=0)

# Check results
print("RAS")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = RAS
  * Nb. elec.   = 4
  * Total spin  = 0.0
  * Active      = [-1 -2 -3 -4 -5]
                = [1 2 3 4 5]
  * 100 configurations

################################################################################

RAS
  > Pass configuration state basis


And lastly, we check the configuration state basis building for RAS model with frozen, and double and empty occupation constraints

In [19]:
# Configuration state basis
PCI.numberElectron = 4
PCI.setConfiguration(mode="RAS",reference=np.array([-1,-2,1,2,3]),active=np.array([-1,-2,-3,1,2,3,4]),
                     frozen=np.array([-1,2]),noDouble=np.array([3]),noEmpty=np.array([3]))

CSB    = np.unique(np.sort(PCI.configuration,axis=1),axis=0)
CSB_th = np.unique(np.sort(np.array([[-1,-2,1,2,3],[-1,-2,4,2,3],[-1,-3,1,2,4]]),axis=1),axis=0)

# Check results
print("RAS with frozen, and double and empty occupation constraints")

if np.all(CSB == CSB_th):
    print("  > Pass configuration state basis")
else:
    print("  > Fail  configuration state basis")

################################################################################
##       Configuration-interaction module for Psi4 with DFT/HF orbitals       ##
################################################################################
  * Author(s):     G. Visentin & F. Mauger
  * Version:       00.01.004
  * Last modified: 04/27/2025

=== Configuration-state basis ==================================================
  * Type        = RAS
  * Nb. elec.   = 5
  * Total spin  = 0.5
  * Active      = [-1 -2 -3]
                = [1 2 3 4]
  * Frozen      = [-1  2]
  * No double   = [3]
  * Not empty   = [3]
  * 3 configurations

################################################################################

RAS with frozen, and double and empty occupation constraints
  > Pass configuration state basis
